# Api Get Commodities


In [ ]:
# Imports
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
# Read credentials
with open('../api_credentials.txt', 'r') as file:
   text = file.read()

credentials = re.findall('=(.*)', text)

In [ ]:
# Assign credentials
client_ID = credentials[0]
client_secret = credentials[1]

In [ ]:
# Create access token
def create_access_token(client_ID, client_secret, region = 'eu'):
   data = {'grant_type': 'client_credentials'}
   response = requests.post('https://%s.battle.net/oauth/token' % region, data = data, auth = (client_ID, client_secret))
   return response.json()

response = create_access_token(client_ID, client_secret)
access_token = response['access_token']

In [ ]:
# Get commodity auctions data
def get_commodities(access_token):
   search = f'https://eu.api.blizzard.com/data/wow/auctions/commodities?namespace=dynamic-eu&locale=en_GB&access_token={access_token}'
   response = requests.get(search)
   return response.json()['auctions']

commodities_data = get_commodities(access_token)
commodities_df = pd.DataFrame(commodities_data)

In [ ]:
# Extract item ids
commodities_df['item'] = commodities_df['item'].apply(lambda x: x['id'])

In [ ]:
# Rename columns
commodities_df.rename(
   columns = {'id': 'auction_id', 'item': 'item_id'}, inplace = True)

In [ ]:
# Tansform prices into x.xx of golds
commodities_df['unit_price'] = commodities_df['unit_price'] / 10000

In [ ]:
# Severite Ore statistical summary
severite_ore = commodities_df[commodities_df['item_id'] == 190396]

severite_ore.describe()

In [ ]:
# Severite Ore remove outliers
Q1 = np.percentile(severite_ore['unit_price'], 25)
Q3 = np.percentile(severite_ore['unit_price'], 75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

severite_ore_outliers_removed = severite_ore[(severite_ore['unit_price'] >= lower_bound) & (severite_ore['unit_price'] <= upper_bound)]

In [ ]:
# Severite Ore price distribution
sns.set_style('darkgrid')
sns.histplot(data = severite_ore_outliers_removed, x = 'unit_price', bins = 20, kde = True, color = 'purple')

plt.xlabel('Price per Ore')
plt.ylabel('Number of Auctions')
plt.title('Serevite Ore Price Distribution')

plt.show()

In [ ]:
# Remove outliers from all items
def remove_outliers(df):
   df_clean = pd.DataFrame()
   for item_id in df['item_id'].unique():
      df_item = df[df['item_id'] == item_id]
      Q1 = df_item['unit_price'].quantile(0.25)
      Q3 = df_item['unit_price'].quantile(0.75)
      IQR = Q3 - Q1
      df_clean_item = df_item[(df_item['unit_price'] >= Q1 - 1.5 * IQR) & (df_item['unit_price'] <= Q3 + 1.5 * IQR)]
      df_clean = pd.concat([df_clean, df_clean_item])
   return df_clean

commodities_wo_outliers = remove_outliers(commodities_df)

In [ ]:
# Group items by price and count auctions + quantities
commodities_counted = commodities_wo_outliers.groupby(['item_id', 'unit_price']).agg(
   auction_count = ('quantity', 'count'), total_quantity = ('quantity', 'sum')).reset_index()

In [ ]:
# Calculate the weighted average 'unit_price' (weights: 'auction_count' + 'total_quantity')
def weighted_average(x):
   return (x['unit_price'] * x['auction_count'] * x['total_quantity']).sum() / (x['auction_count'] * x['total_quantity']).sum()

commodities_weighted = commodities_counted.groupby('item_id').apply(weighted_average)

In [ ]:
# Group by 'item_id' and merge weigthed average prices
commodities_final = commodities_counted.groupby('item_id').agg({'unit_price': np.mean, 'auction_count': np.sum, 'total_quantity': np.sum}).reset_index()
commodities_final = commodities_final.merge(commodities_weighted.rename('weighted'), on = 'item_id')
commodities_final = commodities_final[['item_id', 'auction_count', 'total_quantity', 'unit_price', 'weighted']]

In [ ]:
# Add timestamp
now = datetime.datetime.now()
date_str = now.strftime('%Y-%m-%d %H:%M:%S')
commodities_final['timestamp'] = pd.to_datetime(date_str)

In [ ]:
commodities_final